## Necesary conditions for Knapsack problem
### Install pyQUBO from Recruit Communications Co. Ltd.
    pip install pyqubo
### Install openJij from Jij Inc.  (startup from Tohoku University)
    pip install -U cmake (in google collaboratory, update cmake)
    pip install open jij

# Solve Knapsack problem

### import pyQUBO, openJij, numpy and matplotlib

In [2]:
from pyqubo  import Array,Constraint, Placeholder
import openjij as jij
import numpy as np
import matplotlib.pyplot as plt

Array, Constrains and Placeholders are convenient classes from pyQUBO

### Prepare binary variables

In [3]:
N = 6
Wmax = 3
vartype = "BINARY"
q = Array.create("q",shape=N,vartype=vartype)
y = Array.create("y",shape=Wmax,vartype=vartype)

"q" is name of variables  
shape specifies the shape of variables as vector, matrix, or...  
vartype selects -1 or 1 by "SPIN" and 0 or 1by "BINARY"

In [4]:
print(q)

Array([Binary(q[0]), Binary(q[1]), Binary(q[2]), Binary(q[3]), Binary(q[4]), Binary(q[5])])


In [5]:
print(y)

Array([Binary(y[0]), Binary(y[1]), Binary(y[2])])


### Prepare values and weights

In [10]:
c = np.random.randint(1,N,N)
w = np.random.randint(1,N,N)
k = np.linspace(1,Wmax,Wmax)

### Define cost function

In [12]:
E1 = Constraint((np.dot(w,q)-np.dot(k,y))**2,"weight")

In [19]:
E2 = Constraint((np.sum(y)-1)**2,"y")

In [20]:
E3 = - np.dot(c,q)

In [21]:
Lam1 = Placeholder('Lam1')
Lam2 = Placeholder('Lam2')
E_cost = Lam1*E1+Lam2*E2+E3

### Compile the cost function

In [22]:
model = E_cost.compile()

### Get qubo matrix

In [29]:
feed_dict = {'Lam1': 5.0,'Lam2': 10.0}
Q, offset = model.to_qubo(feed_dict=feed_dict)

### Prepare simulation of quantum annealing

In [30]:
sampler = jij.SQASampler(beta=10.0, gamma=1.0, trotter=4, step_length=10, step_num=10, schedule=None, iteration=1)

This is done by quantum Monte-Carlo simulation  
gamma = strength of quantum fluctuation  
iteration = number of reads  
step_num = number of MCS  
trotter = Trotter number  
step_length = length of MCS in the same gamma   

### Let's simulate quantum annealing

In [31]:
response = sampler.sample_qubo(Q)

### Show results

In [32]:
print(response)

iteration : 1, minimum energy : -16.0, var_type : BINARY
indices: ['q[2]', 'y[0]', 'q[3]', 'y[2]', 'q[1]', 'q[5]', 'y[1]', 'q[0]', 'q[4]'] 
minmum energy state sample : [1, 0, 0, 1, 1, 0, 0, 0, 0]


### minimum sample

In [33]:
response.samples[0]

{'q[2]': 1,
 'y[0]': 0,
 'q[3]': 0,
 'y[2]': 1,
 'q[1]': 1,
 'q[5]': 0,
 'y[1]': 0,
 'q[0]': 0,
 'q[4]': 0}

### decode solution through pyQUBO

In [35]:
dsol, broken, energy = model.decode_solution(response.samples[0], feed_dict = feed_dict, vartype=vartype)

In [39]:
print(dsol["q"])

{2: 1, 3: 0, 1: 1, 5: 0, 0: 0, 4: 0}


In [37]:
print(dsol["y"])

{0: 0, 2: 1, 1: 0}
